In [9]:
import os
import sys

sys.path.append('../..')

import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
from PIL import Image
from torch.nn import functional as F

from torch.utils.data import Subset
from PIL import Image, ImageFont, ImageDraw

from train_utils import Evaluator, Trainer, train, train_certified_wm
from tqdm import tqdm

from data_utils2 import DataUtils, generate_adv_trigger, generate_random_trigger

import torch
import numpy as np

In [16]:
data_utils = DataUtils(
    dataset_name='cifar10',
    root_dir='./data',
    trigger_size=100
)
data_utils.save_image(incre_train_size=0.1, cache_path='../Watermarking/data/cifar-10-python.tar.gz')

Using downloaded and verified file: ./data/CIFAR10/cifar-10-python.tar.gz
Extracting ./data/CIFAR10/cifar-10-python.tar.gz to ./data/CIFAR10
Files already downloaded and verified
Save data to:


./data/CIFAR10/with_trigger/trigger_clean: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 19182.73it/s]


In [6]:
data_utils = DataUtils(
    dataset_name='cifar10_8020',
    root_dir='./data',
    trigger_size=100,
    train_ratio=0.95,
)
data_utils.save_image(incre_train_size=0.2, cache_path='../Watermarking/data/cifar-10-python.tar.gz')

Using downloaded and verified file: ./data/CIFAR10_8020/cifar-10-python.tar.gz
Extracting ./data/CIFAR10_8020/cifar-10-python.tar.gz to ./data/CIFAR10_8020
Files already downloaded and verified
Save data to:


./data/CIFAR10_8020/with_trigger/trigger_clean: 100%|██████████████████████████████| 100/100 [00:00<00:00, 18779.91it/s]


In [10]:
data_utils = DataUtils(
    dataset_name='cifar10_7030',
    root_dir='./data',
    trigger_size=100,
    train_ratio=0.95,
)
data_utils.save_image(incre_train_size=0.3, cache_path='../Watermarking/data/cifar-10-python.tar.gz')

Using downloaded and verified file: ./data/CIFAR10_7030/cifar-10-python.tar.gz
Extracting ./data/CIFAR10_7030/cifar-10-python.tar.gz to ./data/CIFAR10_7030
Files already downloaded and verified
Save data to:


./data/CIFAR10_7030/with_trigger/trigger_clean: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 18692.86it/s]


In [45]:
data_utils = DataUtils(
    dataset_name='cifar10_7525',
    root_dir='./data',
    trigger_size=100,
    train_ratio=0.95,
)
data_utils.save_image(incre_train_size=0.25, cache_path='../Watermarking/data/cifar-10-python.tar.gz')

Exception: Dataset name is invalid

In [11]:
# generate_random_trigger('./data/CIFAR10')
# generate_random_trigger('./data/CIFAR10_8020')
generate_random_trigger('./data/CIFAR10_7030')

In [12]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18().cuda()
    y = net(torch.randn(1, 3, 32, 32).cuda())
    print(y.size())

test()

torch.Size([1, 10])


In [11]:
# from resnet import ResNet18
# net = ResNet18()
# net.load_state_dict(torch.load('../Watermarking/checkpoint/Tu/resnet18_exp2.ckpt'))
# generate_adv_trigger(net, './data/CIFAR10/')

In [12]:
watermarkset = datasets.ImageFolder('./data/CIFAR10/with_trigger/trigger_random/')


In [14]:
from resnet import ResNet18
net = ResNet18()
net.load_state_dict(torch.load('../Watermarking/checkpoint/Tu/resnet18_exp2.ckpt'))
# generate_adv_trigger(net, './data/CIFAR10/')
generate_adv_trigger(net, './data/CIFAR10_7030/')

Accuracy on clean set: 74.0
Accuracy on adv set: 21.0


In [25]:
data_utils = DataUtils(
    dataset_name='cifar10',
    root_dir='./data',
    trigger_size=100,
    train_ratio=1
    
)
data_utils.save_image(incre_train_size=0.1, cache_path='../Watermarking/data/cifar-10-python.tar.gz')
generate_random_trigger('./data/CIFAR10')
net = ResNet18()
net.load_state_dict(torch.load('../Watermarking/checkpoint/Tu/resnet18_exp2.ckpt'))
generate_adv_trigger(net, './data/CIFAR10/')

Using downloaded and verified file: ./data/CIFAR10/cifar-10-python.tar.gz
Extracting ./data/CIFAR10/cifar-10-python.tar.gz to ./data/CIFAR10
Files already downloaded and verified
Save data to:


./data/CIFAR10/with_trigger/trigger_clean: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 19634.42it/s]


Accuracy on clean set: 74.0
Accuracy on adv set: 21.0


In [26]:
data_utils = DataUtils(
    dataset_name='CIFAR10_8020',
    root_dir='./data',
    trigger_size=100,
    train_ratio=1
)
data_utils.save_image(incre_train_size=0.20, cache_path='../Watermarking/data/cifar-10-python.tar.gz')
generate_random_trigger('./data/CIFAR10_8020')
net = ResNet18()
net.load_state_dict(torch.load('../Watermarking/checkpoint/Tu/resnet18_exp2.ckpt'))
generate_adv_trigger(net, './data/CIFAR10_8020/')

Using downloaded and verified file: ./data/CIFAR10_8020/cifar-10-python.tar.gz
Extracting ./data/CIFAR10_8020/cifar-10-python.tar.gz to ./data/CIFAR10_8020
Files already downloaded and verified
Save data to:


./data/CIFAR10_8020/with_trigger/trigger_clean: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 18123.42it/s]


Accuracy on clean set: 74.0
Accuracy on adv set: 21.0


In [15]:
data_utils = DataUtils(
    dataset_name='CIFAR10_7030',
    root_dir='./data',
    trigger_size=100,
    train_ratio=1
)
data_utils.save_image(incre_train_size=0.3, cache_path='../Watermarking/data/cifar-10-python.tar.gz')
generate_random_trigger('./data/CIFAR10_7030')
net = ResNet18()
net.load_state_dict(torch.load('../Watermarking/checkpoint/Tu/resnet18_exp2.ckpt'))
generate_adv_trigger(net, './data/CIFAR10_7030/')

Using downloaded and verified file: ./data/CIFAR10_7030/cifar-10-python.tar.gz
Extracting ./data/CIFAR10_7030/cifar-10-python.tar.gz to ./data/CIFAR10_7030
Files already downloaded and verified
Save data to:


./data/CIFAR10_7030/with_trigger/trigger_clean: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 19305.46it/s]


Accuracy on clean set: 74.0
Accuracy on adv set: 21.0


In [16]:
data_utils = DataUtils(
    dataset_name='CIFAR10_6040',
    root_dir='./data',
    trigger_size=100,
    train_ratio=1
)
data_utils.save_image(incre_train_size=0.4, cache_path='../Watermarking/data/cifar-10-python.tar.gz')
generate_random_trigger('./data/CIFAR10_6040')
net = ResNet18()
net.load_state_dict(torch.load('../Watermarking/checkpoint/Tu/resnet18_exp2.ckpt'))
generate_adv_trigger(net, './data/CIFAR10_6040/')

Using downloaded and verified file: ./data/CIFAR10_6040/cifar-10-python.tar.gz
Extracting ./data/CIFAR10_6040/cifar-10-python.tar.gz to ./data/CIFAR10_6040
Files already downloaded and verified
Save data to:


./data/CIFAR10_6040/with_trigger/trigger_clean: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 5905.81it/s]


Accuracy on clean set: 74.0
Accuracy on adv set: 21.0


In [17]:
data_utils = DataUtils(
    dataset_name='CIFAR10_5050',
    root_dir='./data',
    trigger_size=100,
    train_ratio=1
)
data_utils.save_image(incre_train_size=0.5, cache_path='../Watermarking/data/cifar-10-python.tar.gz')
generate_random_trigger('./data/CIFAR10_5050')
net = ResNet18()
net.load_state_dict(torch.load('../Watermarking/checkpoint/Tu/resnet18_exp2.ckpt'))
generate_adv_trigger(net, './data/CIFAR10_5050/')

Using downloaded and verified file: ./data/CIFAR10_5050/cifar-10-python.tar.gz
Extracting ./data/CIFAR10_5050/cifar-10-python.tar.gz to ./data/CIFAR10_5050
Files already downloaded and verified
Save data to:


./data/CIFAR10_5050/with_trigger/trigger_clean: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 802.51it/s]


Accuracy on clean set: 74.0
Accuracy on adv set: 21.0
